# Lecture 7: Python for Intrument Control (1.5 Hours) #

## Unitful Computing with Pint and python-quantities (10 Minutes)##

To finish the lecture, we'll briefly mention how to use Python to perform numerical computations in a *unitful* way. We'll demonstrate using python-quantities, which provides a *quantity* data type to represent arrays with associated units. In Lecture 7, we'll see a much more flexible example backed by the Pint library.

For now, let's go on and import python-quantities.

In [11]:
import numpy as np
import quantities as pq

We can now create quantities by specifying both a magnitude and a unit string.

In [4]:
velocity = pq.Quantity(2, 'm / s')
velocity

array(2) * m/s

Arithmetic operations on quantities respect units.

In [5]:
acceleration = velocity / pq.second
acceleration

array(2.0) * m/s**2

Importantly, operations on quantities must be correct for dimensions, but may use different units.

In [7]:
pq.Quantity(1, 'meter') + pq.Quantity(1, 'foot')

array(1.3048) * m

In [14]:
try:
    pq.Quantity(1, 'meter') + pq.Quantity(1, 'second')
except ValueError as ex:
    print(ex)

Unable to convert between units of "s" and "m"


Quanities also support transforming between different unit systems; such conversions are also checked for consistent dimensions.

In [8]:
acceleration.rescale('ft / minute**2')

array(23622.047244094487) * ft/min**2

In [15]:
try:
    acceleration.rescale('kg')
except ValueError as ex:
    print(ex)

Unable to convert between units of "m/s**2" and "kg"


Finally, python-quantities also provides a range of useful physical constants that we can use to quickly construct quantities.

In [12]:
Sz = pq.constants.hbar * np.array([[1, 0], [0, -1]]) / 2
H = pq.constants.gamma_p * pq.Quantity(7, 'tesla') * Sz
H

array([[ 3.5,  0. ],
       [ 0. , -3.5]]) * gamma_p*(h/(2*pi))*T

In [13]:
(H.simplified / (2 * np.pi * pq.constants.hbar)).simplified.rescale('MHz')

array([[ 149.02118732,    0.        ],
       [   0.        , -149.02118732]]) * MHz

## More on class inheritance (15 Minutes) ##

## Abstract device component classes (15 Minutes) ##

- channels, I/O, axes

## Let's build up an Instrument and talk to it! (40 Minutes) ##

- like catherine's [tutoral](http://catherineh.github.io/programming/2016/08/15/contributing-a-scientific-instrument-driver-to-instrumentkit.html), is there a real physical device we can use or does a dummy need written

## Unit testing (10 Minutes) ##

- so you dont have a bad day or make anyone else's bad

## Lots of options of starting points, don't reinvent the wheel! ## 

- InstrumentKit
- Instrumental (Mabuchi)
- QuDi (Ulm, Nanodiamonds)